=== Импорт пакетов, добавление переменных, инициализация клиента ===

In [1]:
import os
import json
from openai import OpenAI

OPENAI_API_KEY="MY_API_KEY"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

client = OpenAI()

=== Создание тренировочных данных ===

In [8]:
raw_list = [
    {
        "category_name": "Electronics",
        "description": "Latest gadgets, smartphones, laptops, and accessories."
    },
    {
        "category_name": "Home Appliances",
        "description": "Essentials for your kitchen and household, from refrigerators to microwaves."
    },
    {
        "category_name": "Fashion",
        "description": "Trendy clothing, shoes, and accessories for men, women, and children."
    },
    {
        "category_name": "Beauty & Health",
        "description": "Skincare, makeup, vitamins, and health products for a better lifestyle."
    },
    {
        "category_name": "Toys & Games",
        "description": "Fun and educational toys for children of all ages."
    },
    {
        "category_name": "Books",
        "description": "A wide range of genres including fiction, non-fiction, and educational material."
    },
    {
        "category_name": "Sports & Outdoors",
        "description": "Equipment and apparel for all your sporting needs and outdoor adventures."
    },
    {
        "category_name": "Automotive",
        "description": "Car accessories, tools, and parts for all vehicle needs."
    },
    {
        "category_name": "Pet Supplies",
        "description": "Everything you need for your furry friends, from food to toys."
    },
    {
        "category_name": "Office Supplies",
        "description": "Stationery, furniture, and tech for your workspace."
    },
    {
        "category_name": "Groceries",
        "description": "Daily essentials, fresh produce, and household items."
    }
]

ozon_list = list(raw_list)

print(ozon_list[0])

=== Создание тренировочного файла ===

Тренировочный файл должен содержать не менее 10 примеров (строк).

In [11]:
for message in ozon_list:
    
    ozon_dict = {"messages": []}
    user_message = {"role": "user", "content": f"Write a description of {message['category_name']} category"}
    assist_message = {"role": "assistant", "content": f"{message['description']}"}

    ozon_dict["messages"].append(user_message)
    ozon_dict["messages"].append(assist_message)

    with open("ozonDataset2", 'a') as f:
        f.write(json.dumps(ozon_dict) + "\n")

=== Импорт файла для обучения ===

Этот запрос возвращает FileObject.id идентификатор, который понадобится для создания процесса обучения (джоб). 

In [12]:
client.files.create(
  file=open("ozonDataset2", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-WlvM62mvoDWw75E6C29f66c0', bytes=2121, created_at=1729655826, filename='ozonDataset2', object='file', purpose='fine-tune', status='processed', status_details=None)

=== Создание задачи на обучение ===

Передаются параметры: идентификатор файла для обучения, базовая модель, кастомное имя модели

Этот запрос возвращает FineTuningJob.id идентификатор джобы (процесса обучения). Он понадобится далее.

In [13]:
client.fine_tuning.jobs.create(
  training_file="file-WlvM62mvoDWw75E6C29f66c0",
  model="gpt-4o-mini-2024-07-18",
  suffix="avatar-model"
)

FineTuningJob(id='ftjob-r2j4nHvzzoXYRSgZNw6Ui3xJ', created_at=1729655839, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-5zkGYfg93jnJSHOJv7w6rXCb', result_files=[], seed=1387770150, status='validating_files', trained_tokens=None, training_file='file-WlvM62mvoDWw75E6C29f66c0', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='avatar-model')

=== Получение данных о процессе обучения ===

Передаются параметры: идентификатор джобы (процесса обучения)

Необходимо дождаться, когда статус джобы будет status='succeeded', тогда этот запрос вернёт новое имя кастмной модели "fine_tuned_model"

In [27]:
client.fine_tuning.jobs.retrieve("ftjob-r2j4nHvzzoXYRSgZNw6Ui3xJ")

FineTuningJob(id='ftjob-r2j4nHvzzoXYRSgZNw6Ui3xJ', created_at=1729655839, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal:avatar-model:ALMvv8Xd', finished_at=1729656404, hyperparameters=Hyperparameters(n_epochs=9, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-5zkGYfg93jnJSHOJv7w6rXCb', result_files=['file-HlssQ8gCuecPikJuFhemAY3A'], seed=1387770150, status='succeeded', trained_tokens=2817, training_file='file-WlvM62mvoDWw75E6C29f66c0', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='avatar-model')

=== Тестовый запрос к обученной модели ===

In [26]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="ft:gpt-4o-mini-2024-07-18:personal:avatar-model:ALMvv8Xd",
  messages=[
    {"role": "user", "content": test_request}
  ]
)
print(completion.choices[0].message.content)

Latest gadgets, smartphones, laptops, and accessories.


In [25]:
test_request = "Write a description of Electronics category"